- "Guardrails" as discussed so far seem to encapsulate a validation/verification step & (optionally) a retry step.


- But actually, we may not want this 1st step. Consider one generation forward pass (complete a prompt => parse a completion).
    1. The forward pass succeeds. We get a completion, but want to evaluate the completion. This may involve hardcoded logic or look like the Constiutional AI approach, but note it may not be necessary at all to split this out into separate "check" and "fix" steps. Maybe we just want `guardrail.evaluate(prompt, completion)` or `guardrail.evaluate(input, output)` to handle both.
    2. There's exceptional control flow. E.g. `PydanticOutputParser` raises an exception trying to `json.decode` a completion string or `pydantic.from_json` json object. In this case, the validation step is already done...  we only have to decide whether to retry.
        - Note here how encapsulation gets broken. An `OutputParser` just accepts a completion. But to retry an `OutputParser`, a flexible/general approach would require both the prompt + completion. This is shown here:

In [1]:
from pydantic import BaseModel, Field
from typing import List

from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate


class FloatArray(BaseModel):
    values: List[float] = Field(description="list of floats")

float_array_query = "Write out a few terms of fiboacci."


model = OpenAI(model_name='text-curie-001', temperature=0.5)
parser = PydanticOutputParser(pydantic_object=FloatArray)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

prompt_str = prompt.format_prompt(query=float_array_query).to_string()

print('== PROMPT: '.ljust(40, '='))
print(prompt_str)

completion_str = model(prompt_str)
print('== COMPLETION: '.ljust(40, '='))
print(completion_str)

try:
    parsed_completion = parser.parse(completion_str)
    print('== PARSE: '.ljust(40, '='))
    print(parsed_completion)

# Note: PydanticOutputParser raises very clear exceptions: json.decode fails or pydantic.from_json fails.
# These can be wrapped in/replaced with a passing specific exception.
except Exception as e:
    print('PARSE EXCEPTION!')

== PROMPT: =============================
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below. For example, the object {"foo": ["bar", "baz"]} conforms to the schema {"foo": {"description": "a list of strings field", "type": "string"}}.

Here is the output schema:
```
{"values": {"description": "list of floats", "type": "array"}}
```
Write out a few terms of fiboacci.

== COMPLETION: =========================

The Fibonacci sequence is a sequence of numbers in which each number is the sum of the previous two numbers. The first number in the sequence is 0 and the second number is 1. The Fibonacci sequence is named after Leonardo Fibonacci, an Italian mathematician who first described it in the 12th century.
PARSE EXCEPTION!


In [2]:
more_powerful_model = OpenAI(model_name='text-davinci-003', temperature=0.5)

retry_prompt = PromptTemplate(
    template="Prompt:\n{prompt}\nCompletion:\n{completion}\n\nAbove, the Completion failed to satisfy the given Prompt. Please answer the user query correctly:",
    input_variables=["prompt", "completion"]
)
parser = PydanticOutputParser(pydantic_object=FloatArray)

retry_prompt_str = retry_prompt.format_prompt(prompt=prompt_str, completion=completion_str).to_string()
print('== PROMPT: '.ljust(40, '='))
print(retry_prompt_str)

retry_completion_str = more_powerful_model(retry_prompt_str)
print('== COMPLETION: '.ljust(40, '='))
print(retry_completion_str)

retry_parsed_completion = parser.parse(retry_completion_str)
print('== PARSE: '.ljust(40, '='))
print(retry_parsed_completion)

== PROMPT: =============================
Prompt:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below. For example, the object {"foo": ["bar", "baz"]} conforms to the schema {"foo": {"description": "a list of strings field", "type": "string"}}.

Here is the output schema:
```
{"values": {"description": "list of floats", "type": "array"}}
```
Write out a few terms of fiboacci.

Completion:

The Fibonacci sequence is a sequence of numbers in which each number is the sum of the previous two numbers. The first number in the sequence is 0 and the second number is 1. The Fibonacci sequence is named after Leonardo Fibonacci, an Italian mathematician who first described it in the 12th century.

Above, the Completion failed to satisfy the given Prompt. Please answer the user query correctly:
== COMPLETION: =========================


{"values": [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]}
== PARSE: ==============================
values=[0.0, 1.0,